In [ ]:
# conver the grand truth data (ro confirmed excel to json)

In [52]:
import pandas as pd
import json

# Read the Excel file
df = pd.read_excel("/Users/weng/Library/CloudStorage/OneDrive-SharedLibraries-WBG/Hanoi3 Files - Intaigent/mobile_learning_content/blf_chatbot_script_share.xlsx", sheet_name="Sheet_ALP_share")

# Identify 'Text' columns
text_columns = [col for col in df.columns if col.startswith('Text')]

# Identify all other columns as id_vars
id_vars = [col for col in df.columns if col not in text_columns]

# Melt the DataFrame
df_melted = df.melt(id_vars=id_vars,
                    value_vars=text_columns,
                    var_name='text_num',
                    value_name='Text')

# rename text_num to mobile_summary_number, Text to mobile_summary_final
df_melted.rename(columns={'text_num': 'mobile_summary_number', 'Text': 'mobile_summary_final'}, inplace=True)

# make the mobile_summary_number as integer, remove text
df_melted['mobile_summary_number'] = df_melted['mobile_summary_number'].str.replace('Text', '').astype(int)

# clean variables names to lower case and replace space with underscore
df_melted.columns = df_melted.columns.str.lower().str.replace(' ', '_')

# make course_id as integer
df_melted['course_id'] = df_melted['course_id'].astype(int)

# sort course_id, mobile_summary_number
df_melted.sort_values(['course_id', 'mobile_summary_number'], inplace=True)

# remove the row if mobile_summary_final is empty
df_melted = df_melted[df_melted['mobile_summary_final'].notnull()]

In [53]:
df_melted.columns

Index(['learning_path_(h1)', 'category_(h2)', 'course_(h3)', 'course_id',
       'encouraging_words', 'topic_introduction', 'mobile_summary_number',
       'mobile_summary_final'],
      dtype='object')

In [ ]:
# convert this df_melted to json, following the exact structure of the file blf_mobile_intaigent_english.json. 
# the relationship would be group by course_id, then by mobile_summary_number 'learning_path_(h1)', 'category_(h2)', 'encouraging_words'



In [54]:
# Convert df_melted to the desired JSON structure
result = {}

for _, row in df_melted.iterrows():
    learning_path = row['learning_path_(h1)']
    category = row['category_(h2)']
    course = row['course_(h3)']
    course_id = row['course_id']
    encouraging_words = row['encouraging_words']
    topic_introduction = row['topic_introduction']
    mobile_summary_number = row['mobile_summary_number']
    mobile_summary_final = row['mobile_summary_final']

    if learning_path not in result:
        result[learning_path] = {}
    
    if category not in result[learning_path]:
        result[learning_path][category] = {}
    
    if course not in result[learning_path][category]:
        result[learning_path][category][course] = {
            topic_introduction: {
                encouraging_words: []
            }
        }
    
    summary = {
        "mobile_summary_number": str(mobile_summary_number),
        "mobile_summary_final": mobile_summary_final
    }
    
    result[learning_path][category][course][topic_introduction][encouraging_words].append(summary)

# Sort the summaries by mobile_summary_number
for learning_path in result:
    for category in result[learning_path]:
        for course in result[learning_path][category]:
            for topic in result[learning_path][category][course]:
                for encouraging_words in result[learning_path][category][course][topic]:
                    result[learning_path][category][course][topic][encouraging_words].sort(
                        key=lambda x: int(x['mobile_summary_number'])
                    )

# Write the result to a JSON file
import json

with open('../reference/blf_mobile_intaigent_alp_version.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print("JSON file has been created successfully.")

JSON file has been created successfully.
